In [ ]:
import warnings
warnings.filterwarnings('ignore')

import pandas as pd

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.options.display.float_format = '{:,.2f}'.format

dataset_path = '../datasets/raw/'
dataset_base_name = 'OrcamentoDespesa'
available_years = [
    '2014',
    '2015',
    '2016',
    '2017',
    '2018',
    '2019',
    '2020',
    '2021',
    # '2022',  # Disponivel mas incompleto
]
columns = ['EXERCÍCIO', 'CÓDIGO ÓRGÃO SUPERIOR', 'NOME ÓRGÃO SUPERIOR',
       'CÓDIGO ÓRGÃO SUBORDINADO', 'NOME ÓRGÃO SUBORDINADO',
       'CÓDIGO UNIDADE ORÇAMENTÁRIA', 'NOME UNIDADE ORÇAMENTÁRIA',
       'CÓDIGO FUNÇÃO', 'NOME FUNÇÃO', 'CÓDIGO SUBFUNÇÃO', 'NOME SUBFUNÇÃO',
       'CÓDIGO PROGRAMA ORÇAMENTÁRIO', 'NOME PROGRAMA ORÇAMENTÁRIO',
       'CÓDIGO AÇÃO', 'NOME AÇÃO', 'CÓDIGO CATEGORIA ECONÔMICA',
       'NOME CATEGORIA ECONÔMICA', 'CÓDIGO GRUPO DE DESPESA',
       'NOME GRUPO DE DESPESA', 'CÓDIGO ELEMENTO DE DESPESA',
       'NOME ELEMENTO DE DESPESA', 'ORÇAMENTO INICIAL (R$)',
       'ORÇAMENTO ATUALIZADO (R$)', 'ORÇAMENTO EMPENHADO (R$)',
       'ORÇAMENTO REALIZADO (R$)']

"""
Em 2020 houve uma despesa com CATEGORIA ECONÔMICA e GRUPO DE DESPESA inválidos.
Isto faz com que o pandas assuma que a coluna é do tipo `object`, gerando erros na junção dos datasets, 
portanto as colunas 'CÓDIGO CATEGORIA ECONÔMICA' e 'CÓDIGO GRUPO DE DESPESA' serão convertidas para `object`
"""
dtype_conversions = {
    'CÓDIGO CATEGORIA ECONÔMICA': 'object',
    'CÓDIGO GRUPO DE DESPESA': 'object',
}

raw_df = pd.DataFrame(columns=columns)
df_shapes = []

for year in available_years:
    ydf = pd.read_csv(f"{dataset_path}{year}_{dataset_base_name}.csv", sep=';', encoding='Latin-1')
    ydf = ydf.astype(dtype_conversions)

    raw_df = pd.merge(raw_df, ydf, how='outer')
    df_shapes.append(ydf.shape)

In [ ]:
# Checagem simples da quantidade de linhas, para ter mais confiabilidade no dataset final
total_line_count = 0
for shape in df_shapes:
    total_line_count += shape[0]

print(total_line_count, raw_df.shape)
print(total_line_count == raw_df.shape[0])

# Mantém cópia do dataset original por segurança
df = raw_df

In [ ]:
# Remove colunas de CODIGO, pois são redundantes
code_columns = [col for col in df.columns if 'CÓDIGO' in col]
df = df.drop(code_columns, axis=1)

In [ ]:
# Corrige separador decimal
df['ORÇAMENTO INICIAL (R$)'] = df['ORÇAMENTO INICIAL (R$)'].str.replace(',', '.')
df['ORÇAMENTO ATUALIZADO (R$)'] = df['ORÇAMENTO ATUALIZADO (R$)'].str.replace(',', '.')
df['ORÇAMENTO EMPENHADO (R$)'] = df['ORÇAMENTO EMPENHADO (R$)'].str.replace(',', '.')
df['ORÇAMENTO REALIZADO (R$)'] = df['ORÇAMENTO REALIZADO (R$)'].str.replace(',', '.')

# Converte colunas de ORÇAMENTO em float
budget_columns = [col for col in df.columns if 'ORÇAMENTO ' in col]
budget_dtypes_dict = { col:'float' for col in budget_columns }
df = df.astype(budget_dtypes_dict)

In [ ]:
# Renomeia colunas
column_names = {
    'EXERCÍCIO': 'exercicio', 
    'NOME ÓRGÃO SUPERIOR': 'nome_orgao_superior', 
    'NOME ÓRGÃO SUBORDINADO': 'nome_orgao_subordinado', 
    'NOME UNIDADE ORÇAMENTÁRIA': 'nome_unidade_orcamentaria', 
    'NOME FUNÇÃO': 'nome_funcao', 
    'NOME SUBFUNÇÃO': 'nome_subfuncao', 
    'NOME PROGRAMA ORÇAMENTÁRIO': 'nome_programa_orcamentario', 
    'NOME AÇÃO': 'nome_acao', 
    'NOME CATEGORIA ECONÔMICA': 'nome_categoria_economica', 
    'NOME GRUPO DE DESPESA': 'nome_grupo_despesa', 
    'NOME ELEMENTO DE DESPESA': 'nome_elemento_despesa', 
    'ORÇAMENTO INICIAL (R$)': 'orcamento_inicial',
    'ORÇAMENTO ATUALIZADO (R$)': 'orcamento_atualizado',
    'ORÇAMENTO EMPENHADO (R$)': 'orcamento_empenhado',
    'ORÇAMENTO REALIZADO (R$)': 'orcamento_realizado',
}

df.rename(columns=column_names, inplace=True)

In [ ]:
"""
TODO: Lidar valores negativos de orçamento e outros outliers
"""
df.describe(percentiles = [0,0.001, 0.25,0.5,0.75,0.99, 1])

In [ ]:
# Visualização de informações básicas do dataset
print(df.info())
df.sample(5)

In [ ]:
# Exporta o arquivo pré-processado, com dados agregados de todos os anos
from datetime import datetime

output_path = '../datasets/preprocessed/'
t = datetime.now()

df.to_csv(f"{output_path}{dataset_base_name}_{t}.csv", sep=';', encoding='utf-8', index = False)